In [1]:
import sys
sys.path.append("..")   # this allows us to import from sibling directory

from code_to_import.dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from code_to_import.Classifier_module import Classifier
from torch.autograd import Variable


In [2]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
three_layers = False
model_name = "twoLayerModel_weights_gpu_mumu0"
path_to_model = "../models/two_layers/" + model_name

event_type = "mumu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [3]:
if not three_layers:
    # model = 0
    model = Classifier()
    model.load_state_dict(torch.load(path_to_model))
#     model.eval()

In [4]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()

['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [5]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor).requires_grad_(True))

In [6]:
input.requires_grad=True

In [7]:
# optimizer = torch.optim.Adam(model.parameters(), , betas=(opt.b1, opt.b2)) 
# criterion = nn.MSELoss()   # mean squared error loss
criterion = nn.CrossEntropyLoss()


In [8]:
# model.retain_grad()
input.retain_grad()
output = model(input).requires_grad_(True)
output.retain_grad()

loss = criterion(output, target)
loss.retain_grad()
print(loss)

# external_grad = torch.tensor(1.0)
loss.backward()

tensor(0.8273, grad_fn=<NllLossBackward>)


In [9]:
print(input.grad)

tensor([[ 3.5297e-05,  1.0007e-04,  8.8110e-05,  ...,  3.5259e-04,
         -2.9452e-05,  2.1016e-05],
        [ 6.6288e-04,  7.3669e-05,  2.6386e-04,  ...,  2.2433e-04,
          5.5548e-04, -1.0778e-03],
        [-7.4418e-05, -4.7716e-04, -2.6470e-04,  ...,  6.3488e-04,
          3.7120e-04, -1.7020e-03],
        ...,
        [ 2.3224e-04,  1.1974e-04, -7.8513e-05,  ...,  4.2953e-04,
          1.1348e-04, -3.8296e-04],
        [-1.5894e-05,  5.6012e-05, -1.3287e-04,  ...,  8.9980e-05,
          1.6257e-05, -9.2650e-04],
        [-4.5360e-05, -1.7337e-04, -4.5463e-04,  ...,  6.3940e-04,
          4.0214e-04,  9.6345e-04]])


In [10]:
mean_grad = input.grad.detach().numpy().mean(axis=0)
max_grad = np.max(mean_grad)

mean_grad = mean_grad/max_grad
mean_grad

array([-0.28438386, -0.8189718 , -0.04835104,  0.31521562,  0.6152029 ,
       -0.22921108,  0.36500767, -0.03591432,  0.30708304, -0.0831086 ,
       -0.8434104 , -0.04980291, -0.2545509 , -0.1616709 ,  0.12269182,
        0.33911082,  0.41759297, -0.01147292,  0.01835625,  0.05597574,
       -0.3774777 ,  0.14496884, -0.07805367,  0.76827574,  0.1416043 ,
       -0.11610488,  0.7938891 , -0.75478333,  0.3339535 ,  0.18552092,
        0.6636813 ,  0.52090675, -0.6220686 ,  0.5418682 , -0.05199159,
       -0.2618335 , -0.83723456, -0.3331189 ,  0.90699565, -0.35868794,
        0.15078619, -0.06807779,  0.24713817, -0.32372496, -0.04233977,
       -0.04906697, -0.00697902,  0.04457797, -0.10001454, -0.18522263,
        0.11786818,  0.3291395 , -0.31453374, -0.2413899 , -0.09280032,
        0.31504393,  0.11307193, -0.12384463,  0.18185772,  0.45185494,
        0.16281728, -0.04261361, -0.53609306, -0.466931  ,  0.22110124,
        0.64917254, -0.05443611,  0.08752508, -1.7451752 , -1.68

In [11]:
std_grad = np.std(input.grad.detach().numpy(), axis=0)/np.sqrt(input.shape[0])
std_grad

array([3.83903807e-06, 4.46197055e-06, 5.14693465e-06, 5.56694931e-06,
       3.88198805e-06, 4.22369840e-06, 5.83913652e-06, 4.24561722e-06,
       5.25785572e-06, 4.85559349e-06, 5.14985413e-06, 5.70230759e-06,
       3.23403788e-06, 3.24041048e-06, 3.44311525e-06, 3.33453931e-06,
       3.85923204e-06, 3.33291541e-06, 3.43334591e-06, 3.06741435e-06,
       3.72565410e-06, 3.49442644e-06, 3.30913963e-06, 3.66246832e-06,
       4.91897299e-06, 4.29747206e-06, 4.87549687e-06, 5.49376819e-06,
       3.70787620e-06, 3.50598839e-06, 3.73176022e-06, 3.58069337e-06,
       2.49754066e-06, 3.81872223e-06, 2.81900361e-06, 2.84248949e-06,
       5.27466182e-06, 4.29558395e-06, 3.97292160e-06, 4.07390598e-06,
       5.14079693e-06, 2.67618066e-06, 2.96445455e-06, 2.84881207e-06,
       3.03648631e-06, 2.64599521e-06, 3.16418982e-06, 2.66523216e-06,
       2.61390460e-06, 2.67447967e-06, 2.98978307e-06, 3.09422626e-06,
       2.97538259e-06, 2.95425616e-06, 2.73642968e-06, 3.23933591e-06,
      

In [12]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [13]:
for i in mean_grad.argsort()[::-1]:
#     if mean_grad[i] - 2 * r.importances_std[i] > 0:
    print(f"{input_variables[i]:<8}                    "
          f"{mean_grad[i]:.3f}"
          f" +/- {std_grad[i]:.3f}")


nubar_pt                    1.000 +/- 0.000
llbar_delta_eta                    0.907 +/- 0.000
wminusb_delta_eta                    0.794 +/- 0.000
bbarnubar_delta_phi                    0.768 +/- 0.000
wminusb_delta_phi                    0.664 +/- 0.000
wplus_phi                    0.649 +/- 0.000
lbarb_delta_eta                    0.615 +/- 0.000
top_boosted_eta                    0.542 +/- 0.000
ttbar_mass                    0.533 +/- 0.000
wminusbbar_delta_phi                    0.521 +/- 0.000
nu_eta                      0.452 +/- 0.000
lbarb_delta_phi                    0.418 +/- 0.000
lbarnu_delta_eta                    0.365 +/- 0.000
lnubar_delta_phi                    0.339 +/- 0.000
wplusb_delta_phi                    0.334 +/- 0.000
bbar_pt                     0.330 +/- 0.000
l_eta                       0.329 +/- 0.000
lnubar_delta_eta                    0.315 +/- 0.000
b_eta                       0.315 +/- 0.000
bnu_delta_eta                    0.307 +/- 0.000
tbar_phi   

In [14]:
np.save("./results/derivative_importance" + model_name + ".npy", mean_grad)
np.save("./results/derivative_std" + model_name + ".npy", std_grad)